### Extraer ficheros de microdatos de los .zip descargados de la web EFF

In [2]:
import os
import zipfile

# Lista de olas
waves = list(range(2002, 2021, 3))

# Extraer ficheros
for wave in waves:
    for file in os.scandir(os.path.join("data",str(wave))):
        if file.name.endswith(".zip"):
            with zipfile.ZipFile(file.path, mode="r") as archive:
                   archive.extractall(os.path.join("data",str(wave)))

# Homogeneizar nombres de los ficheros de 2002
for i in range(1,6):
    os.rename(r"data/2002/other_sections_2002_imp{}.dta".format(i),
              r"data/2002/otras_secciones_2002_imp{}.dta".format(i))
                    
print("Extracción terminada")

Extracción terminada


Este notebook tiene dos partes, una dedicada a crear los ficheros de Banca Online, y otra a los ficheros de Carga de la deuda. Se puede ejecutar todo seguido para obtener todos los ficheros.

# Banca Online
### Carga inicial y combinación de dataframes, y homogeneización de variables

In [3]:
import numpy as np
import pandas as pd

df_list = []
for w in range(2002, 2021, 3):
    df = pd.concat(
        [pd.read_stata(r"data/{}/databol{}.dta".format(w,i)).assign(imputation=i, wave = w)
         for i in range(1, 6)]
    )
    if w == 2002:
        df = df.rename(columns={'h_number': 'h_2002'})
    if w in [2002, 2005]:
        varlist = ['p2_8', 'p2_32', 'p2_82', 'p4_10', 'p4_33', 'p4_27']
        for var in varlist:
            df[var] = df[var].replace(2,0)
        varlist = ['p2_5', 'p2_8', 'p2_32', 'p2_82', 'p4_10', 'p4_33', 'p4_27']
        vardict = {var:'n'+var for var in varlist}
        df = df.rename(columns=vardict)
        df = df.drop(columns='p2_1')
    if w > 2002:
        renthog = df.columns[df.columns.str.startswith('renthog')].tolist()[0]
        df['renthog'] = df[renthog]
        df = df.drop(columns=renthog)
    df['h_id'] = w*100000 + df['imputation']*10000 + df['h_'+str(w)]
    df = df.drop(columns='h_'+str(w))
    df_list.append(df)

df1 = pd.concat(df_list)

df_list = []
for w in range(2002, 2021, 3):
    for i in range(1,6):
        df = pd.read_stata(r"data/{}/otras_secciones_{}_imp{}.dta".format(w,w,i)).assign(imputation=i, wave = w)
        #df = pd.read_stata(r"data/{}/otras_secciones_{}_imp1.dta".format(w,w)).assign(wave=w)
        if w == 2002:
            df = df.rename(columns={'h_number': 'h_2002'})
            df['anno'] = np.where(df['anno']==1, 2002, 2003)
        if w in [2002, 2005]:
            df['p1_2d_1'] = df['anno']-df['p1_2b_1']
            df = df.drop(columns='renthog')
        # Identificador wave & imputation & h_wave
        df['h_id'] = w*100000 + df['imputation']*10000 + df['h_'+str(w)]
        #df['h_id'] = df['h_id'] + df['h_'+str(w)]
        df = df.drop(columns='h_'+str(w))
        cols = ["h_id", "p1_2b_1", "p1_2d_1", "p1_5_1", "p8_1", "p8_2", "p8_3", "p8_4",
                "p8_5", "p8_6", "p8_7", "p8_11", "p8_12", "p8_15", "p8_22", "imputation"]
        df = df.filter(items=cols, axis=1)
        df_list.append(df)

df2 = pd.concat(df_list)

# Merge de dataframes
df_eff = pd.merge(left=df1, right=df2, on=['h_id', 'imputation']).set_index('h_id')
print('Dataframes combinados')

Dataframes combinados


### Transformaciones

In [116]:
# Transformaciones
####################
# Edad Persona de Referencia
df_eff['bagepr'] = pd.cut(df_eff["p1_2d_1"],
                           bins=[0, 35, 45, 55, 65, 75, 150],
                           right = False,
                           labels=["Menor de 35 años", "Entre 35 y 44 años", "Entre 45 y 54 años",
                                   "Entre 55 y 64 años", "Entre 65 y 74 años", "Mayor de 74 años"])
df_eff.rename(columns = {'p1_2d_1':'edadpr'}, inplace = True)

# Nivel educativo Persona de Referencia
df_eff['neducpr'] = pd.cut(df_eff["p1_5_1"],
                           bins=[0, 6, 9, 9999],
                           right = False,
                           labels=["Inferior a bachillerato", "Bachillerato", "Educación Universitaria"])
df_eff['p1_5_1'] = df_eff['p1_5_1'].replace(7.5, 97)

# Función que calcula cohorte de nacimiento PR
def asignar_cohorte(year):
    return max([y for y in list(range(1898,2002,3)) if y <= year])
# Asignar cohorte
df_eff['cohortepr'] = df_eff.apply(lambda row : asignar_cohorte(row['p1_2b_1']), axis = 1)

# Uso de banca online
df_eff['bancaonline'] = np.where(df_eff['p8_22'] == 1, 1, 0)

# Deflactar las variables de renta y riqueza para ponerlas en euros de 2020
def deflactar(value, wave):
    # Función que deflacta las cantidades monetarias para ponerlas a euros de 2020
    deflactor = {
        2002 : 1.34754676657268,
        2005 : 1.22646793452151,
        2008 : 1.14215252998112,
        2011 : 1.06333177560599,
        2014 : 1.04193917674581,
        2017 : 1.01438207474027,
        2020 : 1
    }
    return value*deflactor[wave]

df_eff['renthog'] = df_eff.apply(lambda row : deflactar(row['renthog'], row['wave']), axis = 1)
df_eff['riquezanet'] = df_eff.apply(lambda row : deflactar(row['riquezanet'], row['wave']), axis = 1)
df_eff['riquezabr'] = df_eff.apply(lambda row : deflactar(row['riquezabr'], row['wave']), axis = 1)
df_eff['vdeuda'] = df_eff.apply(lambda row : deflactar(row['vdeuda'], row['wave']), axis = 1)
df_eff['pagodeuda'] = df_eff.apply(lambda row : deflactar(row['pagodeuda'], row['wave']), axis = 1)

# Reemplazamos algunos valores por legibilidad
replace_dict = {
    "percrent": {
        1: "Menor de 20", 2: "Entre 20 y 40", 3: "Entre 40 y 60", 4: "Entre 60 y 80",
        5: "Entre 80 y 90", 6: "Entre 90 y 100"
    },
    "percriq": {
        1: "Menor de 25", 2: "Entre 25 y 50", 3: "Entre 50 y 75", 4: "Entre 75 y 90", 5: "Entre 90 y 100"
    }
}
df_eff = df_eff.replace(to_replace=replace_dict)

#df_eff.to_csv(r'data/data_intermedia.csv')

### Cálculos

In [4]:
#import pandas as pd
#import numpy as np

#df_eff = pd.read_csv(r'data/data_intermedia.csv')

In [103]:
def get_proportions(categories, wave):
    df_eff_wave = df_eff[df_eff['wave'] == wave]
    dfs = []
    # Hogares que usan banca online
    pob = df_eff_wave.groupby(['imputation', 'bancaonline'])["facine3"].sum().reset_index().pivot(
        values="facine3", index='imputation', columns=['bancaonline']).mean(0)
    prop = (pob / pob.sum()) * 100
    df = pd.DataFrame({
        "Categoria": "TODOS LOS HOGARES",
        "Grupo": "TODOS LOS HOGARES",
        "Porcentaje": [prop[1]],
        "Numero de hogares que usan banca online": [pob[1]],
        "Numero de hogares que no usan banca online": [pob[0]],
        "Ola": wave
        })
    dfs.append(df)
    
    for category in categories:
        for group in df_eff_wave[category].unique():
            df_trial = df_eff_wave.loc[df_eff_wave[category] == group]
            pob = df_trial.groupby(['imputation', 'bancaonline'])["facine3"].sum().reset_index().pivot(
                values="facine3", index='imputation', columns=['bancaonline']).mean(0)
            prop = (pob / pob.sum()) * 100
            df = pd.DataFrame({
                "Categoria": category,
                "Grupo": group,
                "Porcentaje": [0 if prop[0] == 100 else prop[1]],
                "Numero de hogares que usan banca online": [0 if pob[0] == pob.sum() else pob[1]],
                "Numero de hogares que no usan banca online": [pob[0]],
                "Ola": wave
                })
            dfs.append(df)
    
    return pd.concat(dfs)

# Calculamos los datos para visualizar
dfs = []
categories = ['bagepr', 'neducpr', 'percrent', 'percriq', 'cohortepr']
for wave in range(2002, 2021, 3):
    df_wave = get_proportions(categories, wave)
    dfs.append(df_wave)

df = pd.concat(dfs)
df = df.replace(
    {'bagepr': 'EDAD DE LA PERSONA DE REFERENCIA',
     'neducpr': 'EDUCACIÓN DE LA PERSONA DE REFENCIA',
     'percrent': 'PERCENTIL DE RENTA',
     'percriq': 'PERCENTIL DE RIQUEZA NETA',
     'cohortepr': 'COHORTE DE NACIMIENTO DE PERSONA DE REFERENCIA'
     })
df_online = df.loc[df['Categoria'] != 'COHORTE DE NACIMIENTO DE PERSONA DE REFERENCIA']
df_online.to_excel('data/data_banca_online.xlsx')
df_online

,Categoria,Grupo,Porcentaje,Numero de hogares que usan banca online,Numero de hogares que no usan banca online,Ola
0,TODOS LOS HOGARES,TODOS LOS HOGARES,7.736357,1.140196e+06,1.359796e+07,2002
0,EDAD DE LA PERSONA DE REFERENCIA,Entre 45 y 54 años,9.504492,2.782632e+05,2.649439e+06,2002
0,EDAD DE LA PERSONA DE REFERENCIA,Entre 35 y 44 años,9.728894,3.096299e+05,2.872951e+06,2002
0,EDAD DE LA PERSONA DE REFERENCIA,Mayor de 74 años,0.681440,1.133077e+04,1.651438e+06,2002
0,EDAD DE LA PERSONA DE REFERENCIA,Entre 65 y 74 años,2.317230,5.774423e+04,2.434207e+06,2002
...,...,...,...,...,...,...
0,PERCENTIL DE RIQUEZA NETA,Entre 50 y 75,68.689900,3.230585e+06,1.472559e+06,2020
0,PERCENTIL DE RIQUEZA NETA,Entre 90 y 100,80.889611,1.525079e+06,3.603040e+05,2020
0,PERCENTIL DE RIQUEZA NETA,Entre 25 y 50,61.987037,2.915567e+06,1.787944e+06,2020
0,PERCENTIL DE RIQUEZA NETA,Menor de 25,58.925076,2.771708e+06,1.932075e+06,2020


In [109]:
# Gráfico alternativo de cohortes
df_cohort = df.loc[df['Categoria'] == 'COHORTE DE NACIMIENTO DE PERSONA DE REFERENCIA']
# Identificar si cohorte es par o impar
df_cohort = df_cohort.rename(columns={'Categoria': 'tipocohorte', 'Grupo': 'Cohorte'})

df_cohort['tipocohorte'] = np.where((df_cohort['Cohorte'] % 2) == 0, 'AÑOS PARES', 'AÑOS IMPARES')
df_cohort['Edad'] = df_cohort['Ola'] - df_cohort['Cohorte']
df_cohort.to_excel(r"data/data_cohorte.xlsx")
df_cohort

,tipocohorte,Cohorte,Porcentaje,Numero de hogares que usan banca online,Numero de hogares que no usan banca online,Ola,Edad
0,AÑOS IMPARES,1955,10.832380,104184.468216,8.576030e+05,2002,47
0,AÑOS PARES,1964,7.591025,86247.559744,1.049931e+06,2002,38
0,AÑOS PARES,1922,0.945017,4399.076197,4.611031e+05,2002,80
0,AÑOS IMPARES,1931,2.443467,19213.792566,7.671193e+05,2002,71
0,AÑOS PARES,1952,6.554952,55340.445708,7.889136e+05,2002,50
...,...,...,...,...,...,...,...
0,AÑOS IMPARES,1991,93.735871,243351.304721,1.626255e+04,2020,29
0,AÑOS IMPARES,1937,20.692483,82003.240486,3.142916e+05,2020,83
0,AÑOS PARES,1994,84.172735,110559.710953,2.078889e+04,2020,26
0,AÑOS IMPARES,1997,58.574407,48874.225162,3.456533e+04,2020,23


## Carga de la deuda
### Dataset de carga de deuda

In [135]:
import pandas as pd

data = pd.read_csv(r'data/eff_data.csv')
data = data[
    (data['elemento'] == 'POBLACION TOTAL') |
    (data['elemento'] == 'MEDIANA DEL RATIO "PAGOS POR DEUDAS / RENTA"') |
    ((data['elemento'] == 'DEUDA TOTAL') & (data['estadistico'] == 'PORCENTAJE DE HOGARES POSEEDORES')) |
    (data['elemento'] == 'HOGARES ENDEUDADOS CON RATIO "PAGOS POR DEUDAS / RENTA" SUPERIOR AL 40%')
]
cols = ['ola', 'elemento', 'categoria', 'valor', 'desglose']
data = data[cols]
data = pd.pivot_table(
    data,
    values='valor',
    index=['categoria', 'ola', 'desglose'],
    columns=['elemento']
    ).reset_index()
data['POBLACION TOTAL'] = data['POBLACION TOTAL']*1000000
data[data['desglose'] == 'NUMERO DE MIEMBROS DEL HOGAR TRABAJANDO'] = data[
    data['desglose'] == 'NUMERO DE MIEMBROS DEL HOGAR TRABAJANDO'].replace(
    {'Ninguno': "Ninguno trabaja",
     'Uno': "Uno trabaja",
     'Dos': "Dos trabajan",
     'Tres o mas': "Tres o mas trabajan"
     })
data = data.rename(columns={
    'DEUDA TOTAL': 'PORCENTAJE DE HOGARES CON DEUDAS',
    'HOGARES ENDEUDADOS CON RATIO "PAGOS POR DEUDAS / RENTA" SUPERIOR AL 40%': 'PORC HOGS ENDEUDADOS CON "PAGOS POR DEUDAS / RENTA" SUP A 40%',
})
data.to_excel(r'data/data_carga_deuda.xlsx')
data

elemento,categoria,ola,desglose,PORCENTAJE DE HOGARES CON DEUDAS,"PORC HOGS ENDEUDADOS CON ""PAGOS POR DEUDAS / RENTA"" SUP A 40%","MEDIANA DEL RATIO ""PAGOS POR DEUDAS / RENTA""",POBLACION TOTAL
0,Bachillerato,2002,EDUCACION DEL CABEZA DE FAMILIA,56.576038,5.715766,14.604691,3826590.0
1,Bachillerato,2005,EDUCACION DEL CABEZA DE FAMILIA,62.443644,10.977871,18.949416,3952987.0
2,Bachillerato,2008,EDUCACION DEL CABEZA DE FAMILIA,63.916004,19.031327,21.288665,4323944.0
3,Bachillerato,2011,EDUCACION DEL CABEZA DE FAMILIA,61.132050,12.618089,19.235152,4313783.0
4,Bachillerato,2014,EDUCACION DEL CABEZA DE FAMILIA,61.047553,13.001948,18.627066,4196372.0
...,...,...,...,...,...,...,...
247,Uno trabaja,2014,NUMERO DE MIEMBROS DEL HOGAR TRABAJANDO,57.891124,17.257519,20.409833,5856904.0
248,Uno,2017,NUMERO DE MIEMBROS DEL HOGAR,32.142957,11.543438,19.132492,4729919.0
249,Uno trabaja,2017,NUMERO DE MIEMBROS DEL HOGAR TRABAJANDO,59.594217,14.309040,18.274859,5886563.0
250,Uno,2020,NUMERO DE MIEMBROS DEL HOGAR,38.306206,16.216551,20.206506,4945637.0
